This week we explores the concept of Compression/recompression.

In [1]:
import numpy as np

def compute_sigma_pc(data, tail_n=5, smooth_window=3):
    sigma = np.asarray(data['sigma'], dtype=float)
    e = np.asarray(data['e'], dtype=float)
    if sigma.size != e.size or sigma.size < 6:
        raise ValueError("Need at least 6 paired points of sigma and e.")

    # Ensure strictly increasing sigma (sort if needed)
    order = np.argsort(sigma)
    sigma = sigma[order]
    e = e[order]

    if np.any(sigma <= 0):
        raise ValueError("All sigma values must be > 0 for log10 transform.")

    # Transform x-axis to log10(sigma)
    log_sigma = np.log10(sigma)

    # Simple moving-average smoothing on e (optional)
    if smooth_window is None or smooth_window < 1:
        smooth_window = 1
    if smooth_window % 2 == 0:
        smooth_window += 1  # force odd

    if smooth_window == 1:
        e_s = e.copy()
    else:
        pad = smooth_window // 2
        e_pad = np.pad(e, pad_width=pad, mode='edge')
        kernel = np.ones(smooth_window) / smooth_window
        e_s = np.convolve(e_pad, kernel, mode='valid')  # same length as e

    # First and second derivatives w.r.t. log_sigma
    de_dlog = np.gradient(e_s, log_sigma)
    d2e_dlog2 = np.gradient(de_dlog, log_sigma)

    # Curvature (magnitude): kappa = |e''| / (1 + e'^2)^{3/2}
    kappa = np.abs(d2e_dlog2) / np.power(1.0 + de_dlog**2, 1.5)

    # Avoid selecting ends (numerical artifacts): ignore first/last point
    if kappa.size > 4:
        kappa_use = kappa[2:-2]
        idx_star_rel = np.argmax(kappa_use)
        idx_star = idx_star_rel + 2
    else:
        idx_star = int(np.argmax(kappa))

    # Point of maximum curvature
    x_star = log_sigma[idx_star]
    e_star = e_s[idx_star]
    m_tan = de_dlog[idx_star]                # slope of tangent at star
    theta = np.arctan(m_tan)                 # angle of tangent vs horizontal
    m_bis = np.tan(theta / 2.0)              # slope of bisector with horizontal

    # Fit virgin compression line on high-stress tail: e = a*log_sigma + b
    tail_n = max(3, min(int(tail_n), e_s.size - 2))
    x_tail = log_sigma[-tail_n:]
    e_tail = e_s[-tail_n:]
    a_vir, b_vir = np.polyfit(x_tail, e_tail, 1)

    # Intersection of bisector and virgin line:
    # e_bis(x) = e_star + m_bis*(x - x_star)
    # e_vir(x) = a_vir * x + b_vir
    # Solve e_star + m_bis*(x - x_star) = a_vir*x + b_vir
    # => e_star - m_bis*x_star = (a_vir - m_bis)*x + b_vir
    denom = (a_vir - m_bis)
    if np.isclose(denom, 0.0):
        # Fallback: if bisector nearly parallel to virgin line, use intersection of tangent & virgin
        x_pc = (e_star - m_tan*x_star - b_vir) / (a_vir - m_tan)
    else:
        x_pc = (e_star - m_bis * x_star - b_vir) / denom

    sigma_pc = 10.0 ** x_pc

    return {
        'sigma_pc': float(sigma_pc),
        'log_sigma_pc': float(x_pc),
        'idx_star': int(idx_star),
        'coeff_virgin': (float(a_vir), float(b_vir)),
        'slope_tangent': float(m_tan),
        'slope_bisector': float(m_bis),
        'log_sigma': log_sigma,
        'e_smooth': e_s
    }




In [2]:
data = {
    'sigma': [12.5, 25, 50, 100, 200, 400, 800],  # kPa
    'e':     [1.02, 0.98, 0.95, 0.92, 0.88, 0.84, 0.80]
}
res = compute_sigma_pc(data, tail_n=5, smooth_window=3)
print(f"sigma_pc ≈ {res['sigma_pc']:.1f} (same units as input sigma)")

sigma_pc ≈ 50.0 (same units as input sigma)
